In [1]:
!pip install pdf2image
!pip install pytesseract
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [2]:
from pdf2image import convert_from_path
import re
import numpy as np
import cv2
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
from glob import glob

In [3]:
pdfs =  glob(f"./*RP*.pdf")
pdfs

['./437-2022 RPC.pdf']

In [4]:
# use this command to install open cv2
# pip install opencv-python

# use this command to install PIL
# pip install Pillow

def mark_region(image_path):
    
    im = cv2.imread(image_path)

    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (9,9), 0)
    thresh = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,30)

    # Dilate to combine adjacent text contours
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (9,9))
    dilate = cv2.dilate(thresh, kernel, iterations=4)

    # Find contours, highlight text areas, and extract ROIs
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    line_items_coordinates = []
    for c in cnts:
        area = cv2.contourArea(c)
        x,y,w,h = cv2.boundingRect(c)

        if y >= 600 and x <= 1000:
            if area > 10000:
                image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
                line_items_coordinates.append([(x,y), (2200, y+h)])

        if y >= 2400 and x<= 2000:
            image = cv2.rectangle(im, (x,y), (2200, y+h), color=(255,0,255), thickness=3)
            line_items_coordinates.append([(x,y), (2200, y+h)])
            

    return line_items_coordinates

In [5]:
!apt-get install poppler-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [6]:
VAL = []
for pdf in pdfs:
    print(pdf,end=' ')
    pages = convert_from_path(pdf,500)[0].save(pdf + ".jpg", "JPEG")

    line_items_coordinates = mark_region(pdf + ".jpg")

    # load the original image
    image = cv2.imread(pdf + ".jpg")

    TEXT=[]
    for line_items_coordinate in line_items_coordinates:
        # get co-ordinates to crop the image
        c = line_items_coordinate

        # cropping image img = image[y0:y1, x0:x1]
        img = image[c[0][1]:c[1][1], c[0][0]:c[1][0]]    

        #plt.figure(figsize=(10,10))
        #plt.imshow(img)

        # convert the image to black and white for better OCR
        ret,thresh1 = cv2.threshold(img,120,255,cv2.THRESH_BINARY)

        text = str(pytesseract.image_to_string(thresh1, config='--psm 6'))
        # pytesseract image to string to get results
        TEXT.append(text)
        #print(text)
    try:
        valor = [re.search('Valor.*',t) for t in TEXT]
        valor = np.unique([v.group() for v in valor if v])[0]
        valor = re.sub('Valor: \$\s?','',valor)
        valor = re.sub('Valor. \$\s?','',valor)
        valor = re.sub('Valor \$\s?','',valor)
        valor = re.sub('\,','.',valor)
        VAL.append(valor)
    except: VAL.append('DESCONOCIDO')
    print(VAL)

./437-2022 RPC.pdf ['19.692.339 COP']
